In [8]:
from transformers import BertModel
from data_dir import pretrained_3kmer_dir
from utils.data_generator import _data_generator_mtl
from multitask_learning import init_model_mtl
import os
dataloader = _data_generator_mtl()
model = init_model_mtl(pretrained_3kmer_dir, head="bert", config=os.path.join('models', 'config', 'mtl.json'))
for step, batch in enumerate(dataloader):
    input_ids, attn_mask, label_prom, label_ss, label_polya = tuple(t for t in batch)
    output = model(input_ids, attn_mask)
    # print(output.keys())
    bert = model.shared_layer
    bert_output = bert(input_ids, attn_mask)[0]
    print(bert_output.shape)
    print(output['prom'].shape)



torch.Size([1, 512, 768])
torch.Size([1, 512, 2])


In [13]:
import torch
loss_fn = torch.nn.CrossEntropyLoss()
pred = torch.tensor([
        [[0.5, 0.25], [0.5, 0.25], [0.5, 0.25]]
    ])
label = torch.tensor([[1, 0, 0]])
for p, l in zip(pred, label):
    loss = loss_fn(
        p,
        l
    )
    print(loss)

tensor(0.6593)


In [20]:
import torch 
loss_fn = torch.nn.CrossEntropyLoss()
pred = torch.tensor([[0.0, 0.9]])
label = torch.tensor([0])
loss = loss_fn(pred, label)
loss

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [3]:
from transformers import BertForMaskedLM
from data_dir import pretrained_3kmer_dir
model = BertForMaskedLM.from_pretrained(pretrained_3kmer_dir)
bert_layer = model.bert
sum(p.numel() for p in bert_layer.parameters() if p.requires_grad)

85503744

In [7]:
mtl_seq = [
    ["ATGC" * 128, 0, 0, 1],
    ["TGCG" * 128, 1, 0, 0],
    ["GACT" * 128, 0, 1, 0],
    ["CACG" * 128, 0, 0, 0],
    ["CCAT" * 128, 0, 0, 0],
]
import pandas as pd
import os
df = pd.DataFrame(mtl_seq, columns=["sequence", "label_prom", "label_ss", "label_polya"])
mtl_sample_csv = os.path.join("sample", "mtl", "sample.csv")
os.makedirs(os.path.dirname(mtl_sample_csv), exist_ok=True)
if os.path.exists(mtl_sample_csv):
    os.remove(mtl_sample_csv)
df.to_csv(mtl_sample_csv, index=False)

In [6]:
from random import randint
import os
import pandas as pd
sequences = ['ATGC' * 128, 'TGAC' * 128, 'GATC' * 128, "AGCC" * 128, "TGGA" * 128]
labels = [''.join(['E' if randint(0, 255) % 2 == 0 else '.' for i in range(len(s))]) for s in sequences]

seq2seq_datasample_csv = os.path.join("sample", "seq2seq", "sample.csv")
os.makedirs(os.path.dirname(seq2seq_datasample_csv), exist_ok=True)
if os.path.exists(seq2seq_datasample_csv):
    os.remove(seq2seq_datasample_csv)

df = pd.DataFrame(
    [[seq, label] for seq, label in zip(sequences, labels)],
    columns=['sequence', 'label']
)
df.to_csv(seq2seq_datasample_csv, index=False)



In [ ]:
from sequential_labelling import init_adamw_optimizer, init_seq2seq_model
from data_dir import pretrained_3kmer_dir
from utils.utils import load_checkpoint

model = init_seq2seq_model(pretrained_3kmer_dir)
optimizer = init_adamw_optimizer(model.parameters())

model
